In [46]:
# Import the necessary library
import os

# Ensure the Kaggle API credentials are set correctly
os.environ['KAGGLE_CONFIG_DIR'] = os.path.expanduser('~/.kaggle')

# Define the competition or dataset you want to download
competition_name = 'rashikrahmanpritom/data-science-job-posting-on-glassdoor'

# Use the Kaggle API to download the dataset
# !kaggle competitions download -c {competition_name}

# If you're downloading a dataset, use:
# dataset_name = 'dataset-owner/dataset-name'
!kaggle datasets download -d {competition_name} --force

# Unzip the dataset if necessary
import zipfile

# Use the correct file path and name
zip_file = '/home/ekirush/Documents/MyPortfolio/Portfolio/DATA CLEANING/data-science-job-posting-on-glassdoor.zip'
with zipfile.ZipFile(zip_file, 'r') as zip_ref:
    zip_ref.extractall('/home/ekirush/Documents/MyPortfolio/Portfolio/DATA CLEANING')


Dataset URL: https://www.kaggle.com/datasets/rashikrahmanpritom/data-science-job-posting-on-glassdoor
License(s): CC0-1.0
100%|███████████████████████████████████████| 1.53M/1.53M [00:02<00:00, 790kB/s]
100%|███████████████████████████████████████| 1.53M/1.53M [00:02<00:00, 743kB/s]


In [47]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import re
from datetime import datetime

In [48]:
data = pd.read_csv('/home/ekirush/Documents/MyPortfolio/Portfolio/DATA CLEANING/Uncleaned_DS_jobs.csv',index_col="index")
# data2 = pd.read_csv('/kaggle/input/data-science-job-posting-on-glassdoor/Cleaned_DS_Jobs.csv')
data.head(10)

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors
index,,,,,,,,,,,,,,
0,Sr Data Scientist,$137K-$171K (Glassdoor est.),Description\n\nThe Senior Data Scientist is re...,3.1,Healthfirst\n3.1,"New York, NY","New York, NY",1001 to 5000 employees,1993,Nonprofit Organization,Insurance Carriers,Insurance,Unknown / Non-Applicable,"EmblemHealth, UnitedHealth Group, Aetna"
1,Data Scientist,$137K-$171K (Glassdoor est.),"Secure our Nation, Ignite your Future\n\nJoin ...",4.2,ManTech\n4.2,"Chantilly, VA","Herndon, VA",5001 to 10000 employees,1968,Company - Public,Research & Development,Business Services,$1 to $2 billion (USD),-1
2,Data Scientist,$137K-$171K (Glassdoor est.),Overview\n\n\nAnalysis Group is one of the lar...,3.8,Analysis Group\n3.8,"Boston, MA","Boston, MA",1001 to 5000 employees,1981,Private Practice / Firm,Consulting,Business Services,$100 to $500 million (USD),-1
3,Data Scientist,$137K-$171K (Glassdoor est.),JOB DESCRIPTION:\n\nDo you have a passion for ...,3.5,INFICON\n3.5,"Newton, MA","Bad Ragaz, Switzerland",501 to 1000 employees,2000,Company - Public,Electrical & Electronic Manufacturing,Manufacturing,$100 to $500 million (USD),"MKS Instruments, Pfeiffer Vacuum, Agilent Tech..."
4,Data Scientist,$137K-$171K (Glassdoor est.),Data Scientist\nAffinity Solutions / Marketing...,2.9,Affinity Solutions\n2.9,"New York, NY","New York, NY",51 to 200 employees,1998,Company - Private,Advertising & Marketing,Business Services,Unknown / Non-Applicable,"Commerce Signals, Cardlytics, Yodlee"
5,Data Scientist,$137K-$171K (Glassdoor est.),About Us:\n\nHeadquartered in beautiful Santa ...,4.2,HG Insights\n4.2,"Santa Barbara, CA","Santa Barbara, CA",51 to 200 employees,2010,Company - Private,Computer Hardware & Software,Information Technology,Unknown / Non-Applicable,-1
6,Data Scientist / Machine Learning Expert,$137K-$171K (Glassdoor est.),Posting Title\nData Scientist / Machine Learni...,3.9,Novartis\n3.9,"Cambridge, MA","Basel, Switzerland",10000+ employees,1996,Company - Public,Biotech & Pharmaceuticals,Biotech & Pharmaceuticals,$10+ billion (USD),-1
7,Data Scientist,$137K-$171K (Glassdoor est.),Introduction\n\nHave you always wanted to run ...,3.5,iRobot\n3.5,"Bedford, MA","Bedford, MA",1001 to 5000 employees,1990,Company - Public,Consumer Electronics & Appliances Stores,Retail,$1 to $2 billion (USD),-1
8,Staff Data Scientist - Analytics,$137K-$171K (Glassdoor est.),Intuit is seeking a Staff Data Scientist to co...,4.4,Intuit - Data\n4.4,"San Diego, CA","Mountain View, CA",5001 to 10000 employees,1983,Company - Public,Computer Hardware & Software,Information Technology,$2 to $5 billion (USD),"Square, PayPal, H&R Block"


In [49]:
# Shape of the dataset
data.shape

(672, 14)

In [50]:
# Column names
data.columns

Index(['Job Title', 'Salary Estimate', 'Job Description', 'Rating',
       'Company Name', 'Location', 'Headquarters', 'Size', 'Founded',
       'Type of ownership', 'Industry', 'Sector', 'Revenue', 'Competitors'],
      dtype='object')

## Exploring,Transforming, and cleaning columns in the dataset

In [51]:
sorted(data['Job Title'].unique())

['(Sr.) Data Scientist -',
 'AI Data Scientist',
 'AI Ops Data Scientist',
 'AI/ML - Machine Learning Scientist, Siri Understanding',
 'Analytics - Business Assurance Data Analyst',
 'Analytics Manager',
 'Analytics Manager - Data Mart',
 'Applied AI Scientist / Engineer',
 'Applied Computer Scientist',
 'Applied Technology Researcher / Data Scientist',
 'Associate Data Scientist',
 'Aviation AI/ML Data Scientist',
 'Big Data Engineer',
 'Business Data Analyst',
 'Business Intelligence Analyst',
 'Business Intelligence Analyst I- Data Insights',
 'COMPUTER SCIENTIST - ENGINEER - RESEARCH COMPUTER SCIENTIST - SIGNAL PROCESSING',
 'COMPUTER SCIENTIST - ENGINEER - RESEARCH COMPUTER SCIENTIST - TRANSPORTATION TECHNOLOGY',
 'Chief Scientist',
 'Clinical Data Analyst',
 'Cloud Data Engineer (Azure)',
 'Computational Behavioral Scientist',
 'Computational Scientist',
 'Computational Scientist, Machine Learning',
 'Computer Scientist 1',
 'Computer Vision / Deep Learning Scientist',
 'Data & M

In [52]:
# A function to filter job titles

def job_title(title):
    title_lower = title.lower()
    if 'data scientist' in title_lower:
        return 'Data scientist'
    elif 'data engineer' in title_lower:
        return 'Data engineer'
    elif 'analyst' in title_lower:
        return 'Analyst'
    elif 'machine learning' in title_lower:
        return 'Machine Learning Engineer'
    elif 'manager' in title_lower:
        return 'Manager'
    elif 'director' in title_lower:
        return 'Director'
    return 'NA'

data['Updated Job Title'] = data['Job Title'].apply(job_title)
# data = data.drop('Job Title',axis=1)

In [53]:
sorted(data['Updated Job Title'].unique())

['Analyst',
 'Data engineer',
 'Data scientist',
 'Director',
 'Machine Learning Engineer',
 'Manager',
 'NA']

In [54]:
# Salary Estimate Columns
sorted(data['Salary Estimate'].unique())

['$101K-$165K (Glassdoor est.)',
 '$105K-$167K (Glassdoor est.)',
 '$110K-$163K (Glassdoor est.)',
 '$112K-$116K (Glassdoor est.)',
 '$122K-$146K (Glassdoor est.)',
 '$124K-$198K (Glassdoor est.)',
 '$128K-$201K (Glassdoor est.)',
 '$137K-$171K (Glassdoor est.)',
 '$138K-$158K (Glassdoor est.)',
 '$141K-$225K (Glassdoor est.)',
 '$145K-$225K(Employer est.)',
 '$212K-$331K (Glassdoor est.)',
 '$31K-$56K (Glassdoor est.)',
 '$56K-$97K (Glassdoor est.)',
 '$66K-$112K (Glassdoor est.)',
 '$69K-$116K (Glassdoor est.)',
 '$71K-$123K (Glassdoor est.)',
 '$75K-$131K (Glassdoor est.)',
 '$79K-$106K (Glassdoor est.)',
 '$79K-$131K (Glassdoor est.)',
 '$79K-$133K (Glassdoor est.)',
 '$79K-$147K (Glassdoor est.)',
 '$80K-$132K (Glassdoor est.)',
 '$87K-$141K (Glassdoor est.)',
 '$90K-$109K (Glassdoor est.)',
 '$90K-$124K (Glassdoor est.)',
 '$91K-$150K (Glassdoor est.)',
 '$92K-$155K (Glassdoor est.)',
 '$95K-$119K (Glassdoor est.)',
 '$99K-$132K (Glassdoor est.)']

In [55]:
data['Salary Estimate'] = data['Salary Estimate'].apply(lambda x:x.replace('(Employer est.)','').replace(' (Glassdoor est.)','').replace('$',''))

In [56]:
sorted(data['Salary Estimate'].unique())

['101K-165K',
 '105K-167K',
 '110K-163K',
 '112K-116K',
 '122K-146K',
 '124K-198K',
 '128K-201K',
 '137K-171K',
 '138K-158K',
 '141K-225K',
 '145K-225K',
 '212K-331K',
 '31K-56K',
 '56K-97K',
 '66K-112K',
 '69K-116K',
 '71K-123K',
 '75K-131K',
 '79K-106K',
 '79K-131K',
 '79K-133K',
 '79K-147K',
 '80K-132K',
 '87K-141K',
 '90K-109K',
 '90K-124K',
 '91K-150K',
 '92K-155K',
 '95K-119K',
 '99K-132K']

In [57]:
# Remove letter K, and multiply values with 1000

def salary_estimate(values):
    data = values.replace('K','')
    data2 = data.split('-')
    data3 = [str(int(datas)*1000) for datas in data2]
    return '-'.join(data3)

data['Salary Estimate'] =  data['Salary Estimate'].apply(salary_estimate)   


In [58]:
sorted(data['Salary Estimate'].unique())

['101000-165000',
 '105000-167000',
 '110000-163000',
 '112000-116000',
 '122000-146000',
 '124000-198000',
 '128000-201000',
 '137000-171000',
 '138000-158000',
 '141000-225000',
 '145000-225000',
 '212000-331000',
 '31000-56000',
 '56000-97000',
 '66000-112000',
 '69000-116000',
 '71000-123000',
 '75000-131000',
 '79000-106000',
 '79000-131000',
 '79000-133000',
 '79000-147000',
 '80000-132000',
 '87000-141000',
 '90000-109000',
 '90000-124000',
 '91000-150000',
 '92000-155000',
 '95000-119000',
 '99000-132000']

In [59]:
# Job Description Column
data['Job Description'][150]

'JOB SUMMARY\nResponsible for designing, analyzing, modeling, and deploying analytical applications to enrich the agent and customer experience in the Agent Portal organization.\n\nMAJOR DUTIES AND RESPONSIBILITIES\nActively and consistently support all efforts to simplify and enhance the customer experience\nUnderstand customer business use cases and be able to translate them to technical specifications and vision on how to implement a solution\nClearly communicate the benefits of the analytic solutions to both a business as well as a technical audience\nLeverage knowledge in analytic and statistical algorithms to help customers explore methods to improve their business\nLead large-scale analytical research projects through all stages including concept formulation, determination of appropriate statistical methodology, data manipulation, research evaluation, and final research report\nDesign, build, and extract large and complex data sets while thinking strategically about uses of data

In [60]:
data['Job Description'] = data['Job Description'].str.replace('•', '.') 
data['Job Description'] = data['Job Description'].str.replace('\n', ' ')
data['Job Description'] = data['Job Description'].str.strip()

data['Job Description'].iloc[3]

"JOB DESCRIPTION:  Do you have a passion for Data and Machine Learning? Do you dream of working with customers on their most forward-looking AI initiatives? Does the challenge of developing modern machine learning solutions to solve real-world manufacturing problems exciting to you?  We develop software for monitoring semiconductor manufacturing process and are looking to leverage the latest technologies to address our customer's needs. You will be part of a team that investigates and builds solutions based all the data available in factories, ranging from time series data, to post manufacturing data, to production logs. You will be working side by side with application developers and customers on real world problems with actual manufacturing data.  JOB FUNCTION:  Basic and applied research in statistical machine learning, deep learning, and data science as well as signal and information processing to advance the state of the art in time series analysis of semiconductor manufacturing d

In [61]:
# Rating Column

sorted(data['Rating'].unique())

[-1.0,
 2.0,
 2.1,
 2.2,
 2.3,
 2.4,
 2.5,
 2.6,
 2.7,
 2.8,
 2.9,
 3.0,
 3.1,
 3.2,
 3.3,
 3.4,
 3.5,
 3.6,
 3.7,
 3.8,
 3.9,
 4.0,
 4.1,
 4.2,
 4.3,
 4.4,
 4.5,
 4.6,
 4.7,
 4.8,
 4.9,
 5.0]

In [62]:
data['Rating'] = np.where(data['Rating'] == -1,0,data['Rating'])
print(sorted(data.Rating.unique()))

[0.0, 2.0, 2.1, 2.2, 2.3, 2.4, 2.5, 2.6, 2.7, 2.8, 2.9, 3.0, 3.1, 3.2, 3.3, 3.4, 3.5, 3.6, 3.7, 3.8, 3.9, 4.0, 4.1, 4.2, 4.3, 4.4, 4.5, 4.6, 4.7, 4.8, 4.9, 5.0]


In [63]:
# 'Company Name' Column

sorted(data['Company Name'].unique())

['1-800-Flowers\n2.7',
 '1010data\n3.1',
 '10x Genomics\n4.2',
 '1904labs\n4.7',
 '22nd Century Technologies\n3.7',
 '7Park Data\n3.9',
 'A Place for Mom\n2.7',
 'A-Line Staffing Solutions\n4.1',
 'ABIOMED\n4.1',
 'ALTA IT Services\n3.9',
 'ASRC Federal Holding Company\n3.4',
 'AccessHope',
 'Acuity Insurance\n4.8',
 'Advance Sourcing Concepts\n3.4',
 'Advanced Bio-Logic Solutions Corp\n4.0',
 'Advanced BioScience Laboratories\n2.7',
 'Adwait Algorithm\n4.4',
 'AeroVironment\n4.2',
 'Affinity Solutions\n2.9',
 'AgreeYa Solutions\n3.8',
 'Alaka`ina Foundation Family of Companies\n3.6',
 'Alector\n4.8',
 'Alignment Healthcare\n3.5',
 'Alion Science & Technology\n3.6',
 'Allen Institute\n3.5',
 'AllianceBernstein\n3.2',
 'Allied Solutions\n3.4',
 'Altus Group\n3.7',
 'Americo Life\n3.3',
 'Ameritas Life Insurance Corp\n3.0',
 'Amyris\n3.3',
 'Analysis Group\n3.8',
 'Apex Systems\n3.9',
 'AppLovin\n4.8',
 'Apple\n4.1',
 'Applied Research Laboratories\n3.8',
 'Aptive\n3.5',
 'Argo Group US\

In [64]:
data['Company Name'] = data['Company Name'].replace(r'\n\d\.\d', '', regex=True)

sorted(data['Company Name'].unique())

['1-800-Flowers',
 '1010data',
 '10x Genomics',
 '1904labs',
 '22nd Century Technologies',
 '7Park Data',
 'A Place for Mom',
 'A-Line Staffing Solutions',
 'ABIOMED',
 'ALTA IT Services',
 'ASRC Federal Holding Company',
 'AccessHope',
 'Acuity Insurance',
 'Advance Sourcing Concepts',
 'Advanced Bio-Logic Solutions Corp',
 'Advanced BioScience Laboratories',
 'Adwait Algorithm',
 'AeroVironment',
 'Affinity Solutions',
 'AgreeYa Solutions',
 'Alaka`ina Foundation Family of Companies',
 'Alector',
 'Alignment Healthcare',
 'Alion Science & Technology',
 'Allen Institute',
 'AllianceBernstein',
 'Allied Solutions',
 'Altus Group',
 'Americo Life',
 'Ameritas Life Insurance Corp',
 'Amyris',
 'Analysis Group',
 'Apex Systems',
 'AppLovin',
 'Apple',
 'Applied Research Laboratories',
 'Aptive',
 'Argo Group US',
 'Arsenal Biosciences',
 'AstraZeneca',
 'Autodesk',
 'Averity',
 'Aveshka, Inc.',
 'Avlino',
 'Axiologic Solutions',
 'B4Corp',
 'BWX Technologies',
 'Battelle',
 'Bayside Solut

In [65]:
# Location Column

sorted(data.Location.unique())

['Adelphi, MD',
 'Akron, OH',
 'Alexandria, VA',
 'Alpharetta, GA',
 'Ann Arbor, MI',
 'Annapolis Junction, MD',
 'Appleton, WI',
 'Arlington, VA',
 'Ashburn, VA',
 'Atlanta, GA',
 'Aurora, CO',
 'Austin, TX',
 'Baltimore, MD',
 'Beavercreek, OH',
 'Bedford, MA',
 'Bellevue, WA',
 'Bethesda, MD',
 'Birmingham, AL',
 'Blue Bell, PA',
 'Boston, MA',
 'Bothell, WA',
 'Boulder, CO',
 'Bridgeport, WV',
 'Brisbane, CA',
 'Brooklyn, NY',
 'Burbank, CA',
 'Burlingame, CA',
 'California',
 'Cambridge, MA',
 'Carmel, IN',
 'Carpinteria, CA',
 'Carson, CA',
 'Cedar Rapids, IA',
 'Champaign, IL',
 'Chandler, AZ',
 'Chantilly, VA',
 'Charlotte, NC',
 'Chicago, IL',
 'Cincinnati, OH',
 'Cleveland, OH',
 'Collegeville, PA',
 'Colorado Springs, CO',
 'Columbia, MD',
 'Columbia, MO',
 'Columbia, SC',
 'Concord, CA',
 'Coral Gables, FL',
 'Culver City, CA',
 'Cupertino, CA',
 'Dallas, TX',
 'Danvers, MA',
 'Dayton, OH',
 'Dearborn, MI',
 'Denver, CO',
 'Durham, NC',
 'Edison, NJ',
 'Emeryville, CA',
 'E

In [66]:
# Regular expression to split the location into city and state
data[['City', 'State']] = data['Location'].str.extract(r'^(.*?),\s*([A-Z]{2})$')

# Drop the original 'location' column if it's no longer needed
data.drop(columns=['Location'], inplace=True)

In [67]:
# Headquarters Column

sorted(data.Headquarters.unique())

['-1',
 'Akron, OH',
 'Albertville, AL',
 'Albuquerque, NM',
 'Alexandria, VA',
 'Aliso Viejo, CA',
 'Alpharetta, GA',
 'Annapolis Junction, MD',
 'Arlington, VA',
 'Armonk, NY',
 'Ashburn, VA',
 'Atlanta, GA',
 'Aurora, CO',
 'Aurora, Canada',
 'Austin, TX',
 'Bad Ragaz, Switzerland',
 'Baltimore, MD',
 'Bangalore, India',
 'Basel, Switzerland',
 'Beavercreek, OH',
 'Bedford, MA',
 'Bellevue, WA',
 'Beltsville, MD',
 'Bengaluru, India',
 'Benicia, CA',
 'Bethesda, MD',
 'Birmingham, AL',
 'Blue Bell, PA',
 'Boston, MA',
 'Brentford, United Kingdom',
 'Brisbane, CA',
 'Bristol, PA',
 'Burlingame, CA',
 'Cambridge, MA',
 'Cambridge, United Kingdom',
 'Carle Place, NY',
 'Carmel, IN',
 'Carpinteria, CA',
 'Carson, CA',
 'Cedar Rapids, IA',
 'Chadds Ford, PA',
 'Chandler, AZ',
 'Chantilly, VA',
 'Charlotte, NC',
 'Chennai, India',
 'Cherry Hill, NJ',
 'Chevy Chase, MD',
 'Chicago, IL',
 'Cincinnati, OH',
 'Cleveland, OH',
 'Clifton Park, NY',
 'Coconut Creek, FL',
 'Columbia, MD',
 'Colum

In [68]:
# data.Headquarters = np.where(data.Headquarters == '-1', 'N/A',data.Headquarters)

data[['Headquarter', 'Headquarters State']] = (data['Headquarters'].str.split(',', expand=True))

data = data.drop('Headquarters',axis=1)

data['Headquarters State'].unique()

array([' NY', ' VA', ' MA', ' Switzerland', ' CA', ' IL', ' MO', ' WA',
       ' NJ', ' DC', ' TN', ' AZ', ' PA', ' GA', ' FL', ' WI', ' NE',
       ' Singapore', ' Japan', ' LA', ' OH', ' IN', ' MD', ' CT',
       ' United Kingdom', ' MN', ' TX', ' UT', ' CO', ' OR', ' MI', ' SC',
       None, ' France', ' Canada', ' Bermuda', ' IA', ' India', ' Israel',
       ' NC', ' AL', ' Sweden', ' 061', ' WV', ' HI', ' Spain', ' NM'],
      dtype=object)

In [69]:
data.Headquarter = data.Headquarter.str.strip()
data['Headquarters State'] = data['Headquarters State'].str.strip()

In [70]:
data['Headquarters State'].unique()

array(['NY', 'VA', 'MA', 'Switzerland', 'CA', 'IL', 'MO', 'WA', 'NJ',
       'DC', 'TN', 'AZ', 'PA', 'GA', 'FL', 'WI', 'NE', 'Singapore',
       'Japan', 'LA', 'OH', 'IN', 'MD', 'CT', 'United Kingdom', 'MN',
       'TX', 'UT', 'CO', 'OR', 'MI', 'SC', None, 'France', 'Canada',
       'Bermuda', 'IA', 'India', 'Israel', 'NC', 'AL', 'Sweden', '061',
       'WV', 'HI', 'Spain', 'NM'], dtype=object)

In [71]:
# Size Column

sorted(data.Size.unique())

['-1',
 '1 to 50 employees',
 '10000+ employees',
 '1001 to 5000 employees',
 '201 to 500 employees',
 '5001 to 10000 employees',
 '501 to 1000 employees',
 '51 to 200 employees',
 'Unknown']

In [72]:
data.Size = np.where(data.Size == '-1','Unknown',data.Size)

In [73]:
# data.Size = data.Size.str.replace(' employees','')
# Size Total employee in that company

data.Size = data.Size.apply(lambda x: x.replace(' to ','-').replace(' employees',''))
print(sorted(data['Size'].unique()))

['1-50', '10000+', '1001-5000', '201-500', '5001-10000', '501-1000', '51-200', 'Unknown']


In [74]:
# Type of ownership column
print(data['Type of ownership'].dtype)
sorted(data['Type of ownership'].unique())

object


['-1',
 'College / University',
 'Company - Private',
 'Company - Public',
 'Contract',
 'Government',
 'Hospital',
 'Nonprofit Organization',
 'Other Organization',
 'Private Practice / Firm',
 'Self-employed',
 'Subsidiary or Business Segment',
 'Unknown']

In [75]:
data['Type of ownership'] = np.where(data['Type of ownership'] == '-1','Unknown',data['Type of ownership'])

In [76]:
def ownership_type(strr):
    if 'Company - Public' in strr:
        return strr.replace('Company - Public','Public Company')
    elif 'Company - Private' in strr:
        return strr.replace('Company - Private', 'Private Company')
    elif 'Private Practice / Firm' in strr:
        return strr.replace('Private Practice / Firm', 'Private Company')
    elif 'Subsidiary or Business Segment' in strr:
        return strr.replace('Subsidiary or Business Segment', 'Business')
    elif 'College / University' in strr:
        return strr.replace('College / University', 'Education')
    return strr

data['Type of ownership'] = data['Type of ownership'].apply(ownership_type)
data['Type of ownership'].unique()

array(['Nonprofit Organization', 'Public Company', 'Private Company',
       'Government', 'Business', 'Other Organization', 'Unknown',
       'Hospital', 'Self-employed', 'Education', 'Contract'], dtype=object)

In [77]:
# Industry Column

print(data.Industry.dtype)
sorted(data.Industry.unique())

object


['-1',
 'Accounting',
 'Advertising & Marketing',
 'Aerospace & Defense',
 'Architectural & Engineering Services',
 'Banks & Credit Unions',
 'Biotech & Pharmaceuticals',
 'Cable, Internet & Telephone Providers',
 'Chemical Manufacturing',
 'Colleges & Universities',
 'Computer Hardware & Software',
 'Construction',
 'Consulting',
 'Consumer Electronics & Appliances Stores',
 'Consumer Products Manufacturing',
 'Department, Clothing, & Shoe Stores',
 'Electrical & Electronic Manufacturing',
 'Energy',
 'Enterprise Software & Network Solutions',
 'Express Delivery Services',
 'Farm Support Services',
 'Federal Agencies',
 'Financial Transaction Processing',
 'Food & Beverage Manufacturing',
 'Food & Beverage Stores',
 'Health Care Services & Hospitals',
 'Health, Beauty, & Fitness',
 'Hotels, Motels, & Resorts',
 'IT Services',
 'Industrial Manufacturing',
 'Insurance Agencies & Brokerages',
 'Insurance Carriers',
 'Internet',
 'Investment Banking & Asset Management',
 'Lending',
 'Logi

In [78]:
data.Industry = np.where(data.Industry == '-1','N/A',data.Industry)

In [79]:
data.Industry = data.Industry.str.replace('&','and')
sorted(data.Industry.unique())

['Accounting',
 'Advertising and Marketing',
 'Aerospace and Defense',
 'Architectural and Engineering Services',
 'Banks and Credit Unions',
 'Biotech and Pharmaceuticals',
 'Cable, Internet and Telephone Providers',
 'Chemical Manufacturing',
 'Colleges and Universities',
 'Computer Hardware and Software',
 'Construction',
 'Consulting',
 'Consumer Electronics and Appliances Stores',
 'Consumer Products Manufacturing',
 'Department, Clothing, and Shoe Stores',
 'Electrical and Electronic Manufacturing',
 'Energy',
 'Enterprise Software and Network Solutions',
 'Express Delivery Services',
 'Farm Support Services',
 'Federal Agencies',
 'Financial Transaction Processing',
 'Food and Beverage Manufacturing',
 'Food and Beverage Stores',
 'Health Care Services and Hospitals',
 'Health, Beauty, and Fitness',
 'Hotels, Motels, and Resorts',
 'IT Services',
 'Industrial Manufacturing',
 'Insurance Agencies and Brokerages',
 'Insurance Carriers',
 'Internet',
 'Investment Banking and Asset 

In [80]:
# Sector Columns
print(data.Sector.dtype)
sorted(data.Sector.unique())

object


['-1',
 'Accounting & Legal',
 'Aerospace & Defense',
 'Agriculture & Forestry',
 'Biotech & Pharmaceuticals',
 'Business Services',
 'Construction, Repair & Maintenance',
 'Consumer Services',
 'Education',
 'Finance',
 'Government',
 'Health Care',
 'Information Technology',
 'Insurance',
 'Manufacturing',
 'Media',
 'Non-Profit',
 'Oil, Gas, Energy & Utilities',
 'Real Estate',
 'Retail',
 'Telecommunications',
 'Transportation & Logistics',
 'Travel & Tourism']

In [81]:
data.Sector = np.where(data.Sector == '-1','N/A',data.Sector)

In [82]:
data.Sector = data.Industry.str.replace('&','and')
sorted(data.Sector.unique())

['Accounting',
 'Advertising and Marketing',
 'Aerospace and Defense',
 'Architectural and Engineering Services',
 'Banks and Credit Unions',
 'Biotech and Pharmaceuticals',
 'Cable, Internet and Telephone Providers',
 'Chemical Manufacturing',
 'Colleges and Universities',
 'Computer Hardware and Software',
 'Construction',
 'Consulting',
 'Consumer Electronics and Appliances Stores',
 'Consumer Products Manufacturing',
 'Department, Clothing, and Shoe Stores',
 'Electrical and Electronic Manufacturing',
 'Energy',
 'Enterprise Software and Network Solutions',
 'Express Delivery Services',
 'Farm Support Services',
 'Federal Agencies',
 'Financial Transaction Processing',
 'Food and Beverage Manufacturing',
 'Food and Beverage Stores',
 'Health Care Services and Hospitals',
 'Health, Beauty, and Fitness',
 'Hotels, Motels, and Resorts',
 'IT Services',
 'Industrial Manufacturing',
 'Insurance Agencies and Brokerages',
 'Insurance Carriers',
 'Internet',
 'Investment Banking and Asset 

In [83]:
# Revenue Column

print(data.Revenue.dtype)
sorted(data.Revenue.unique())

object


['$1 to $2 billion (USD)',
 '$1 to $5 million (USD)',
 '$10 to $25 million (USD)',
 '$10+ billion (USD)',
 '$100 to $500 million (USD)',
 '$2 to $5 billion (USD)',
 '$25 to $50 million (USD)',
 '$5 to $10 billion (USD)',
 '$5 to $10 million (USD)',
 '$50 to $100 million (USD)',
 '$500 million to $1 billion (USD)',
 '-1',
 'Less than $1 million (USD)',
 'Unknown / Non-Applicable']

In [84]:
data.Revenue = np.where(data.Revenue == '-1', 'N/A',data.Revenue)
data.Revenue = np.where(data.Revenue == 'Unknown / Non-Applicable', 'N/A',data.Revenue)

In [85]:
def less_than(value):
    if 'Less than $1 million (USD)' in value:
        return value.replace('$','').replace(' million (USD)','')
    return value

data['Revenue'] = data['Revenue'].apply(less_than)
sorted(data.Revenue.unique())

['$1 to $2 billion (USD)',
 '$1 to $5 million (USD)',
 '$10 to $25 million (USD)',
 '$10+ billion (USD)',
 '$100 to $500 million (USD)',
 '$2 to $5 billion (USD)',
 '$25 to $50 million (USD)',
 '$5 to $10 billion (USD)',
 '$5 to $10 million (USD)',
 '$50 to $100 million (USD)',
 '$500 million to $1 billion (USD)',
 'Less than 1',
 'N/A']

In [86]:
data.Revenue = data.Revenue.apply(lambda x: x.replace('$','').replace(' to ','-').replace(' million (USD)',''))
sorted(data.Revenue.unique())

['1-2 billion (USD)',
 '1-5',
 '10+ billion (USD)',
 '10-25',
 '100-500',
 '2-5 billion (USD)',
 '25-50',
 '5-10',
 '5-10 billion (USD)',
 '50-100',
 '500 million-1 billion (USD)',
 'Less than 1',
 'N/A']

In [87]:
def more_conversion(values):
    if '+ billion (USD)' in values:
        data = values.replace('+ billion (USD)','')
        return str(int(data)*1000)+'+'
    return values

data['Revenue'] = data['Revenue'].apply(more_conversion)
sorted(data.Revenue.unique())

['1-2 billion (USD)',
 '1-5',
 '10-25',
 '100-500',
 '10000+',
 '2-5 billion (USD)',
 '25-50',
 '5-10',
 '5-10 billion (USD)',
 '50-100',
 '500 million-1 billion (USD)',
 'Less than 1',
 'N/A']

In [88]:
def more_conversions(value):
    if '500 million-1 billion (USD)' in value:
        data = value.replace(' million', '').replace(' billion (USD)', '')
        data2 = data.split('-')
        lower_bound = int(data2[0].strip())
        upper_bound = int(float(data2[1].strip()) * 1000)  # Convert billion to million
        bounnd = [str(lower_bound), str(upper_bound)]
        return '-'.join(bounnd)
    return value

data['Revenue'] = data['Revenue'].apply(more_conversions)
sorted(data['Revenue'].unique())

['1-2 billion (USD)',
 '1-5',
 '10-25',
 '100-500',
 '10000+',
 '2-5 billion (USD)',
 '25-50',
 '5-10',
 '5-10 billion (USD)',
 '50-100',
 '500-1000',
 'Less than 1',
 'N/A']

In [89]:
# Function to convert '1-2 billion (USD)' to '1000-2000 million'
def convert_to_millions(value):
    if ' billion (USD)' in value:
        # Remove ' billion (USD)' from the string
        cleaned_value = value.replace(' billion (USD)', '')
        # Split the range if it exists
        range_parts = cleaned_value.split('-')
        # Convert each part from billions to millions
        range_in_millions = [str(int(part) * 1000) for part in range_parts]
        # Join the converted parts with a hyphen
        return '-'.join(range_in_millions)
    return value

# Apply the conversion function to the column
data['Revenue'] = data['Revenue'].apply(convert_to_millions)

sorted(data.Revenue.unique())

['1-5',
 '10-25',
 '100-500',
 '1000-2000',
 '10000+',
 '2000-5000',
 '25-50',
 '5-10',
 '50-100',
 '500-1000',
 '5000-10000',
 'Less than 1',
 'N/A']

In [90]:
# Competitors Columns
sorted(data.Competitors.unique())

['-1',
 'ACRT Services, Bartlett Tree Experts',
 'AT&T, Verizon',
 'Accenture, Deloitte, PwC',
 'Accenture, Northrop Grumman, Xerox',
 'Activision Blizzard, Electronic Arts',
 'Acxiom, Merkle, Epsilon (North Carolina)',
 'Adecco, Manpower',
 'Adecco, ManpowerGroup, Allegis Corporation',
 'Advisory Board, Booz Allen Hamilton, McKinsey & Company',
 'Amazon, Accenture, Microsoft',
 'Amazon, Apple',
 'AppDynamics, Datadog, Dynatrace',
 'Aquent, 24 Seven Talent',
 'Arbella Insurance, Safety Insurance',
 'Archibus, iOffice, Planon',
 'Battelle, General Atomics, SAIC',
 'Bechtel Jacobs, Black & Veatch, HNTB',
 'Booz Allen Hamilton, CACI International',
 'Booz Allen Hamilton, Deloitte, ERPi',
 'Booz Allen Hamilton, SAIC, LMI',
 'Bosch, Lear Corporation, Faurecia',
 'Braintree, Authorize.Net, PayPal',
 'Bromium, FireEye, Authentic8',
 'CDW, PCM, SHI International',
 'CGI (Nevada), Accenture, Deloitte',
 'CSC, ManTech, SAIC',
 'CUNA Mutual, SWBC, Overby-Seawell',
 'Cadence Design Systems, Synops

In [91]:
data = data.drop('Competitors', axis=1)


In [92]:
# Renaming column 'City' to 'Location'

data = data.rename(columns={'City':'Location'})

In [93]:
# Adding columna 'min_salary','max_salary', and 'avg_salary'

data['Min Salary'] = data['Salary Estimate'].apply(lambda x: int(x.split('-')[0]))
data['Max Salary'] = data['Salary Estimate'].apply(lambda x: int(x.split('-')[1]))
# Calculate average salary
data['Average Salary'] = (data['Min Salary'] + data['Max Salary']) / 2

In [94]:
# Create the 'same_state' column
data['Same State'] = np.where(data['State'] == data['Headquarters State'], 'Same State', 'Different State')
data['Same State'].unique()

# data['Headquarters State']

array(['Same State', 'Different State'], dtype=object)

In [95]:
# Founded Year Column

data.Founded.unique()

array([1993, 1968, 1981, 2000, 1998, 2010, 1996, 1990, 1983, 2014, 2012,
       2016, 1965, 1973, 1986, 1997, 2015, 1945, 1988, 2017, 2011, 1967,
       1860, 1992, 2003, 1951, 2005, 2019, 1925, 2008, 1999, 1978, 1966,
       1912, 1958, 2013, 1849, 1781, 1926, 2006, 1994, 1863, 1995,   -1,
       1982, 1974, 2001, 1985, 1913, 1971, 1911, 2009, 1959, 2007, 1939,
       2002, 1961, 1963, 1969, 1946, 1957, 1953, 1948, 1850, 1851, 2004,
       1976, 1918, 1954, 1947, 1955, 2018, 1937, 1917, 1935, 1929, 1820,
       1952, 1932, 1894, 1960, 1788, 1830, 1984, 1933, 1880, 1887, 1970,
       1942, 1980, 1989, 1908, 1853, 1875, 1914, 1898, 1956, 1977, 1987,
       1896, 1972, 1949, 1962])

In [96]:
data.Founded = np.where(data.Founded == -1,2024,data.Founded)

In [97]:
# Create the 'company_age' column

data['company_age'] = (datetime.now().year - data.Founded)
data['company_age'].unique()

array([ 31,  56,  43,  24,  26,  14,  28,  34,  41,  10,  12,   8,  59,
        51,  38,  27,   9,  79,  36,   7,  13,  57, 164,  32,  21,  73,
        19,   5,  99,  16,  25,  46,  58, 112,  66,  11, 175, 243,  98,
        18,  30, 161,  29,   0,  42,  50,  23,  39, 111,  53, 113,  15,
        65,  17,  85,  22,  63,  61,  55,  78,  67,  71,  76, 174, 173,
        20,  48, 106,  70,  77,  69,   6,  87, 107,  89,  95, 204,  72,
        92, 130,  64, 236, 194,  40,  91, 144, 137,  54,  82,  44,  35,
       116, 171, 149, 110, 126,  68,  47,  37, 128,  52,  75,  62])

In [98]:
# Creating 'Python' Column with a bolean value generating data from 'Job Description' column

def python(skill):
    if 'python' in skill.lower():
        return 1
    else:
        return 0
data['Python'] = data['Job Description'].apply(python)

In [99]:
# Creating 'Excel' Column with a bolean value generating data from 'Job Description' column

def excel(skill):
    if 'excel' in skill.lower():
        return 1
    else:
        return 0
data['Excel'] = data['Job Description'].apply(excel)

In [100]:
# Creating 'Hadoop' Column with a bolean value generating data from 'Job Description' column

def hadoop(skill):
    if 'hadoop' in skill.lower():
        return 1
    else:
        return 0
data['Hadoop'] = data['Job Description'].apply(hadoop)

In [101]:
# Creating 'Spark' Column with a bolean value generating data from 'Job Description' column

def spark(skill):
    if 'spark' in skill.lower():
        return 1
    else:
        return 0
data['Spark'] = data['Job Description'].apply(spark)

In [102]:
# Creating 'AWS' Column with a bolean value generating data from 'Job Description' column

def aws(skill):
    if 'aws' in skill.lower():
        return 1
    else:
        return 0
data['AWS'] = data['Job Description'].apply(aws)

In [103]:
# Creating 'Tableau' Column with a bolean value generating data from 'Job Description' column

def tableau(skill):
    if 'tableau' in skill.lower():
        return 1
    else:
        return 0
data['Tableau'] = data['Job Description'].apply(tableau)

In [104]:
# Creating 'Big Data' Column with a bolean value generating data from 'Job Description' column

def big_data(skill):
    if 'big data' in skill.lower():
        return 1
    else:
        return 0
data['Big Data'] = data['Job Description'].apply(big_data)

In [105]:
# Creating 'Job Type' Column with a bolean value of the job type

def Job_type(title):
    if 'sr' in title.lower() or 'senior' in title.lower() or 'sr' in title.lower() or 'lead' in title.lower() or 'principal' in title.lower():
            return 'Senior'
    elif 'jr' in title.lower() or 'jr.' in title.lower():
        return 'Junior'
    else:
        return 'NA'
    
data['Job Type'] = data['Job Title'].apply(Job_type)

In [106]:
data['Job Type'].unique()

array(['Senior', 'NA', 'Junior'], dtype=object)

In [107]:
# Reset index so to drop index column

data = data.reset_index()

In [108]:
data = data.drop('index',axis=1)

In [109]:
data.duplicated().sum()

13

In [110]:
data.shape

(672, 29)

In [111]:
data = data.drop_duplicates()

In [112]:
data.shape

(659, 29)

In [113]:
data.describe()

,Rating,Founded,Min Salary,Max Salary,Average Salary,company_age,Python,Excel,Hadoop,Spark,AWS,Tableau,Big Data
count,659.000000,659.000000,659.000000,659.000000,659.000000,659.000000,659.000000,659.000000,659.000000,659.000000,659.000000,659.000000,659.000000
mean,3.651593,1990.494689,99066.767830,148464.339909,123765.553869,33.505311,0.726859,0.447648,0.215478,0.268589,0.247344,0.185129,0.209408
std,1.090806,40.054498,33266.334795,48387.743556,39907.282895,40.054498,0.445911,0.497629,0.411466,0.443562,0.431796,0.388697,0.407195
min,0.000000,1781.000000,31000.000000,56000.000000,43500.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.400000,1982.500000,79000.000000,119000.000000,103000.000000,10.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,3.800000,2003.000000,91000.000000,133000.000000,114000.000000,21.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,4.300000,2014.000000,122000.000000,165000.000000,136500.000000,41.500000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000
max,5.000000,2024.000000,212000.000000,331000.000000,271500.000000,243.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [114]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 659 entries, 0 to 671
Data columns (total 29 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Job Title           659 non-null    object 
 1   Salary Estimate     659 non-null    object 
 2   Job Description     659 non-null    object 
 3   Rating              659 non-null    float64
 4   Company Name        659 non-null    object 
 5   Size                659 non-null    object 
 6   Founded             659 non-null    int64  
 7   Type of ownership   659 non-null    object 
 8   Industry            659 non-null    object 
 9   Sector              659 non-null    object 
 10  Revenue             659 non-null    object 
 11  Updated Job Title   659 non-null    object 
 12  Location            637 non-null    object 
 13  State               637 non-null    object 
 14  Headquarter         659 non-null    object 
 15  Headquarters State  639 non-null    object 
 16  Min Salary   

In [115]:
# Trimming string data in the dataset
data = data.apply(lambda x: x.strip() if isinstance(x,str) else x)

In [116]:
data.head()

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Size,Founded,Type of ownership,Industry,Sector,...,Same State,company_age,Python,Excel,Hadoop,Spark,AWS,Tableau,Big Data,Job Type
0,Sr Data Scientist,137000-171000,Description The Senior Data Scientist is resp...,3.1,Healthfirst,1001-5000,1993,Nonprofit Organization,Insurance Carriers,Insurance Carriers,...,Same State,31,0,0,0,0,1,0,0,Senior
1,Data Scientist,137000-171000,"Secure our Nation, Ignite your Future Join th...",4.2,ManTech,5001-10000,1968,Public Company,Research and Development,Research and Development,...,Same State,56,0,0,1,0,0,0,1,NA
2,Data Scientist,137000-171000,Overview Analysis Group is one of the larges...,3.8,Analysis Group,1001-5000,1981,Private Company,Consulting,Consulting,...,Same State,43,1,1,0,0,1,0,0,NA
3,Data Scientist,137000-171000,JOB DESCRIPTION: Do you have a passion for Da...,3.5,INFICON,501-1000,2000,Public Company,Electrical and Electronic Manufacturing,Electrical and Electronic Manufacturing,...,Different State,24,1,1,0,0,1,0,0,NA
4,Data Scientist,137000-171000,Data Scientist Affinity Solutions / Marketing ...,2.9,Affinity Solutions,51-200,1998,Private Company,Advertising and Marketing,Advertising and Marketing,...,Same State,26,1,1,0,0,0,0,0,NA


In [117]:
data.to_csv('job_posting_Glassdoor.csv',index=False)